# PantryPalML: Production Inference Notebook

This notebook demonstrates real inference using the saved production model and the `ProductionRecipeScorer` API.

It shows:
- Environment setup (Colab-friendly)
- Loading saved model + metadata
- Fetching user interactions with `UserInteractionFetcher`
- Generating top-N recommendations with `ProductionRecipeScorer`
- Basic validation and smoke tests


In [1]:
# Colab/Local environment setup (silent if local)
import sys, subprocess, os, pathlib

IN_COLAB = "google.colab" in sys.modules
repo_root = pathlib.Path.cwd()

if IN_COLAB:
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                        "lightgbm", "pandas", "numpy", "scikit-learn"],
                       check=False)
    except Exception as e:
        print(f"pip install warning: {e}")

    if not (repo_root / "recipe_recommender").exists():
        subprocess.run(["git", "-c", "advice.detachedHead=false", "clone", "-q", "https://github.com/marcel-qayoom-taylor/PantryPalML.git"], check=True)
        os.chdir("PantryPalML")
        repo_root = pathlib.Path.cwd()

print(f"Environment ready. Project root: {repo_root}")


Environment ready. Project root: /Users/marcelqayoomtaylor/Documents/GitHub/PantryPalML/notebooks


In [2]:
# Central paths + defaults (output/input/model, weights, params)
from recipe_recommender.config import get_ml_config

# Streams/filters combined_events.csv to user-level interaction list for recipes
from recipe_recommender.etl.fetch_user_interactions import UserInteractionFetcher

# Loads saved LightGBM + metadata, builds user profile features, scores all recipes
from recipe_recommender.inference.recipe_scorer import RecipeScorer

config = get_ml_config()
print("Model dir:", config.model_dir)
print("Output dir:", config.output_dir)


Model dir: /Users/marcelqayoomtaylor/Documents/GitHub/PantryPalML/recipe_recommender/output/hybrid_models
Output dir: /Users/marcelqayoomtaylor/Documents/GitHub/PantryPalML/recipe_recommender/output


### Load Model and Generate Recommendations
We load the saved LightGBM booster and metadata via `ProductionRecipeScorer` and score recipes for a user.


In [3]:
# Fetch interactions and run inference
fetcher = UserInteractionFetcher(config)

# Example: pick a user with interactions (you can replace with a known user_id)
sample_user_id = 'afcacbe1-eaba-415f-b03e-14ed682af65e'

# # Quickly sample a user from events file
# import pandas as pd
# try:
#     events_path = config.output_dir / "combined_events.csv"
#     if events_path.exists():
#         df_head = pd.read_csv(events_path)
#         # value_counts over recipe-related events finds an active user candidate
#         candidates = (
#             df_head[df_head["event"].isin(fetcher.recipe_events)]["distinct_id"].value_counts()
#         )
#         if len(candidates) > 0:
#             sample_user_id = candidates.index[0]
# except Exception as e:
#     print("Warning reading events head:", e)

if sample_user_id is None:
    sample_user_id = "demo_user_123"  # fallback for demo

print("Using user:", sample_user_id)
# Returns list[dict] with event_type, timestamp, optional recipe_id
interactions = fetcher.fetch_user_interactions(sample_user_id)

# Loads saved booster + metadata, builds user profile features, scores all recipes
scorer = RecipeScorer(config)
# Returns ranked recommendations with scores and metadata
result = scorer.get_user_recipe_recommendations(sample_user_id, interactions, n_recommendations=10)

# Display top results (robust to empty results)
import pandas as pd
recs_df = pd.DataFrame(result.get("recommendations", []))[:10]
if recs_df.empty:
    print("No recommendations returned.")
else:
    display(recs_df[["recipe_id", "recipe_name", "score"]])


2025-09-29 18:02:16,314 - recipe_recommender.etl.fetch_user_interactions - INFO - Initialized UserInteractionFetcher
2025-09-29 18:02:16,314 - recipe_recommender.etl.fetch_user_interactions - INFO -    Events file: /Users/marcelqayoomtaylor/Documents/GitHub/PantryPalML/recipe_recommender/output/combined_events.csv
2025-09-29 18:02:16,314 - recipe_recommender.etl.fetch_user_interactions - INFO -    Tracking 8 event types
2025-09-29 18:02:16,315 - recipe_recommender.etl.fetch_user_interactions - INFO - Fetching interactions for user: afcacbe1-eaba-415f-b03e-14ed682af65e
2025-09-29 18:02:16,315 - recipe_recommender.etl.fetch_user_interactions - INFO - Reading events file in chunks
2025-09-29 18:02:16,422 - recipe_recommender.etl.fetch_user_interactions - INFO -    Processed 50,000 rows, found 0 matches...
2025-09-29 18:02:16,459 - recipe_recommender.etl.fetch_user_interactions - INFO - Found 74 recipe interactions for user afcacbe1-eaba-415f-b03e-14ed682af65e
2025-09-29 18:02:16,460 - rec

Using user: afcacbe1-eaba-415f-b03e-14ed682af65e


2025-09-29 18:02:16,576 - recipe_recommender.inference.recipe_scorer - INFO - Generated scores for 1967 recipes
2025-09-29 18:02:16,576 - recipe_recommender.inference.recipe_scorer - INFO -    Score range: -0.0871 - -0.0405
2025-09-29 18:02:16,576 - recipe_recommender.inference.recipe_scorer - INFO -    No threshold; selecting by top-N
2025-09-29 18:02:16,577 - recipe_recommender.inference.recipe_scorer - INFO -    Returning top 10 recommendations


,recipe_id,recipe_name,score
0,1228,Mexican Shrimp,-0.040458
1,1232,Instant Pot Chicken Mole,-0.040458
2,1264,Carrot Cake Muffins,-0.040458
3,1338,Crock Pot Chicken Tortilla Soup,-0.040458
4,1440,Slow Cooker Butter Chicken,-0.040458
5,1483,Pumpkin Monkey Bread,-0.040458
6,1537,Chile Verde Pork,-0.040458
7,1589,Healthy Cranberry Apricot Bars,-0.040458
8,1601,Moroccan Meatballs,-0.040458
9,1606,Instant Pot Vegetarian Chili,-0.040458


In [4]:
# Smoke tests
errors = []

# 1) Model artifacts exist
model_file = config.model_dir / "hybrid_lightgbm_model.txt"
meta_file = config.model_dir / "hybrid_lightgbm_metadata.json"
if not model_file.exists():
    errors.append(f"Missing model file: {model_file}")
if not meta_file.exists():
    errors.append(f"Missing metadata file: {meta_file}")

# 2) Recipe feature file exists
recipe_file = config.output_dir / "enhanced_recipe_features_from_db.csv"
if not recipe_file.exists():
    errors.append(f"Missing recipe features file: {recipe_file}")

# 3) Recommendation output sanity
if len(result.get("recommendations", [])) == 0:
    errors.append("No recommendations returned")

if errors:
    print("SMOKE TEST: FAIL")
    for e in errors:
        print(" - ", e)
else:
    print("SMOKE TEST: PASS")
    print(f"Top recommendation: {recs_df.iloc[0]['recipe_name']} (score={recs_df.iloc[0]['score']:.4f})")


SMOKE TEST: PASS
Top recommendation: Mexican Shrimp (score=-0.0405)
